# Watson Visual Recognition Training with Spectrogram Images from SETI Signal Data

* https://www.ibm.com/watson/developercloud/visual-recognition/api/v3/
* https://www.ibm.com/watson/developercloud/doc/visual-recognition/customizing.html
* https://github.com/watson-developer-cloud/python-sdk
* https://github.com/watson-developer-cloud/python-sdk/blob/master/watson_developer_cloud/visual_recognition_v3.py

<hr>

## Install the Watson Developer Cloud Python SDK

* Install the Python SDK if has not been previously installed
* Restart the kernel, after installing the SDK

In [1]:
#!pip install --user --upgrade watson-developer-cloud

In [2]:
#Making a local folder to put my data.

### SET YOUR TEAM NAME HERE! Use this folder to save intermediate results
teamname = "Machine_Contact_Enterprise_SETI"
istrainingset = False

mydatafolder = os.path.join( os.environ['PWD'], teamname )  
if os.path.exists(mydatafolder) is False:
    raise Exception('This folder needs to already exist. See the Build_And_Package_Spectrograms_HackathonSpark notebook')

zipfilefolder = mydatafolder + '/zippedPngsTestFinal_4'
if os.path.exists(zipfilefolder) is False:
    raise Exception('This folder needs to already exist. See the Build_And_Package_Spectrograms_HackathonSpark notebook')

In [3]:
from watson_developer_cloud import VisualRecognitionV3
apiVer = VisualRecognitionV3.latest_version #'2016-05-20'
classifier_prefix = teamname + "_v1_"

#You can sign up with WatsonVR through Bluemix to get a key
#However, Hackathon participants will be provided with a WATSON VR key that has more free API calls per day.
apiKey = 'c70486e04c0e6ac12314da71f2fdb88992420c33'  


In [4]:
import glob
import json
import os
import time
import timeit
import zipfile

from random import randint

import numpy as np

import re



<br/>
## Init the Watson Visual Recognition Python Library

* you may need to install the SDK first: **`!pip install --upgrade watson-developer-cloud`**
* you will need the **API** key from the Watson Visual Recognition Service

In [5]:
vr = VisualRecognitionV3(apiVer, api_key=apiKey)

<br/>
## Look For Existing Custom Classifier
Use an existing custom classifier (and update) if one exists, else a new custom classifier will be created

In [6]:
classifier_id = None
classifier = None

classifiers = vr.list_classifiers()

for c in classifiers['classifiers']:
    if c['status'] == 'ready' and (classifier_prefix in c['classifier_id']):
        classifier_id = c['classifier_id']


if classifier_id is not None:
    classifier = vr.get_classifier(classifier_id)
    print '\r\nFound classifer:\r\n\r\n{}'.format(json.dumps(classifier, indent=2))
else:
    print 'No custom classifier available\r\n'
    print(json.dumps(classifiers, indent=2))

No custom classifier available

{
  "classifiers": [
    {
      "status": "ready", 
      "classifier_id": "Machine_Contact_Enterprise_SET_1466269118", 
      "name": "Machine_Contact_Enterprise_SETI_v1_"
    }, 
    {
      "status": "ready", 
      "classifier_id": "feedo_elwin_v1__1756785049", 
      "name": "feedo_elwin_v1_"
    }, 
    {
      "status": "ready", 
      "classifier_id": "SpinFlip_v1__348338520", 
      "name": "SpinFlip_v1_"
    }, 
    {
      "status": "ready", 
      "classifier_id": "team_magicai_v6_296521247", 
      "name": "team_magicai_v6"
    }, 
    {
      "status": "ready", 
      "classifier_id": "sanjoy_v_sanjoy__1846021728", 
      "name": "sanjoy_v_sanjoy_"
    }, 
    {
      "status": "ready", 
      "classifier_id": "team_magicai_v4_87105057", 
      "name": "team_magicai_v4"
    }, 
    {
      "status": "failed", 
      "classifier_id": "team_magicai_v3_reshape_262024401", 
      "name": "team_magicai_v3_reshape"
    }, 
    {
      "status": 

In [7]:
#zz = zipfile.ZipFile('Machine_Contact_Enterprise_SETI/zippedPngsFinal_1' + '/' + 'testset_1.zip')
#test_list = zz.namelist()
#randomSignal = zz.open(test_list[4],'r')

In [8]:
#ls -l Machine_Contact_Enterprise_SETI/zippedPngsTestFinal_1

In [9]:
#classifier_id = "Machine_Contact_Enterprise_SET_1466269118"
#classifier_id = "Machine_Contact_Enterprise_SET_842846996"
#classifier_id = "Machine_Contact_Enterprise_SET_549328565", 
classifier_id = "Machine_Contact_Enterprise_SET_778931065"




#note - have to 'open' this again because it was already .read() out in the line above
#randomSignal = zz.open(test_list[10],'r')

#url_result = vr.classify(images_file=randomSignal, classifier_ids=classifier_id, threshold=0.0)

#print(json.dumps(url_result, indent=2))

## Run the Complete Test Set

In [10]:
#Create a dictionary object to store results from Watson

from collections import defaultdict

class_list = ['squiggle', 'noise', 'narrowband', 'narrowbanddrd']

results_group_by_class = {}
for classification in class_list:
    results_group_by_class[classification] = defaultdict(list)
    
failed_to_classify_uuid_list = []

print classifier_id

results_group_by_class


Machine_Contact_Enterprise_SET_778931065


{'narrowband': defaultdict(list, {}),
 'narrowbanddrd': defaultdict(list, {}),
 'noise': defaultdict(list, {}),
 'squiggle': defaultdict(list, {})}

In [11]:
# locate test archives that were produced in step 3 and add them to the test set
test_set = []
for classification in class_list:
    test_set = np.concatenate((test_set, sorted(glob.glob('{}/testset_*.zip'.format(zipfilefolder)))))

In [12]:
#ls -lah Machine_Contact_Enterprise_SETI/

In [13]:
testresults = []      

for image_archive_name in test_set:
    image_count = 0
    # count number of images in <image_archive_name>
    with zipfile.ZipFile(image_archive_name,'r') as image_archive:
        images = image_archive.namelist()
        image_count = len(images)    
        
    # bulk classify images in <image_archive_name>
    with open(image_archive_name, 'rb') as images_file:
        print 'Running test ({} images) for {}.'.format(image_count, image_archive_name)
        classify_results = vr.classify(images_file=images_file, classifier_ids=[classifier_id], threshold=0.0)

        for classify_result in classify_results['images']:
            pngfilename = classify_result['image'].split('/')[-1]
            uuid = pngfilename.split('.')[0]

            if "error" in classify_result:
                # print error information
                print classify_result
                #add to failed list
                failed_to_classify_uuid_list.append(uuid)                  
            else:
                classifiers_arr = classify_result['classifiers']
                element = [uuid]
                score_list = []
                for classifier_result in classifiers_arr:
                    for class_result in classifier_result['classes']:
                        score_list.append((class_result['class'],class_result['score']))
                #sort alphabetically
                score_list.sort(key = lambda x: x[0])
                score_list = map(lambda x:x[1], score_list)      
                testresults.append(element + score_list)
print "DONE."

Running test (20 images) for /gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/zippedPngsTestFinal_4/testset_1.zip.
Running test (20 images) for /gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/zippedPngsTestFinal_4/testset_10.zip.
Running test (20 images) for /gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/zippedPngsTestFinal_4/testset_11.zip.
Running test (20 images) for /gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/zippedPngsTestFinal_4/testset_12.zip.
{u'image': u'/gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/zippedPngsTestFinal_4/testset_12.zip/gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/cleanPngsTestFinal/9799c20d-88cd-4613-b6c3-10aa66d1845e.dat.png', u'error': {u'error_id': u'server_error', u'descr

KeyboardInterrupt: 

In [14]:
import sys
import csv

fwriter = csv.writer(sys.stdout, delimiter=',')
print len(testresults)
for items in testresults:
    fwriter.writerow(items)

1787
00096d4c-7e03-4c9b-b5e0-671900604308,0.227515,0.780697,0.0744361,0.0507461
016f4745-7434-43b6-9a33-58d43cbc2678,0.320455,0.638881,0.0952986,0.0476314
01cc5d4a-82e4-49ed-97fa-b0bfc033b68c,0.440679,0.478709,0.0545468,0.0507859
02437a5e-7eb2-4963-ae90-b77bce703c39,0.40608,0.434338,0.0691262,0.0791889
0252d7fb-2f7f-49d6-8584-1a6f78f5828e,0.217997,0.783203,0.0496744,0.0478787
0278448b-2db7-42ee-b124-cff7540b3676,0.638133,0.31751,0.0639282,0.0583442
02a1214c-abdf-4372-b205-3fb730318820,0.683341,0.242445,0.0599295,0.0501618
02d2fd3b-6006-41c5-9948-87b2de845633,0.74239,0.231136,0.0683468,0.052384
040a2245-bf91-4d35-9f4a-39bf6dc1786e,0.932516,0.0521397,0.197568,0.0463926
040d13fc-beee-4e9a-aea2-ad52e92de53a,0.630895,0.367866,0.0586867,0.0457893
052d3af1-18af-417f-85a7-abee64c3c594,0.29152,0.636639,0.0690966,0.0579777
0bc8e558-e27c-4591-b570-13ae85d83377,0.474536,0.54448,0.0530906,0.0475085
0c7f7a96-5c45-41af-91c2-903534102714,0.506175,0.2963,0.0485858,0.0657115
0f12f064-bc99-471b-b128-bcf5